In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/melon_song.csv')
df.head(1)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201.0,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138267,햇살은 우릴 위해 내리고 \r\n바람도 서롤 감싸게 했죠 \r\n우리 웃음속에 계절...,445029956 411111859


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6247 entries, 0 to 6246
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   songId     6247 non-null   int64  
 1   title      6247 non-null   object 
 2   artist     6247 non-null   object 
 3   genre      6245 non-null   object 
 4   album      6247 non-null   object 
 5   lyricist   5849 non-null   object 
 6   composer   5845 non-null   object 
 7   date       6202 non-null   float64
 8   img        6247 non-null   object 
 9   comment    6247 non-null   int64  
 10  like       6247 non-null   int64  
 11  lyric      6247 non-null   object 
 12  plylstSeq  6126 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 634.6+ KB


In [4]:
df.fillna('', inplace=True)

In [5]:
df['comment_like_total'] = df.comment + df.like

In [6]:
df['songId'] = df.songId.astype(str)

### 형태소 분석과 불용어 처리

In [7]:
with open('data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]
stop_words.extend('은 는 를 도 을 며 의 에 게 니 거 로 요 과 래 랑 파 여 에게'.split())

In [8]:
from konlpy.tag import Okt
okt = Okt()

In [9]:
%%time
reviews = []
for review in df.lyric:
    review = review.replace('\n', ' ')
    morphs = okt.morphs(review, stem=True)
    tmp = [word for word in morphs  if word not in stop_words]
    reviews.append(' '.join(tmp))
df['morphs'] = reviews

CPU times: total: 2min 7s
Wall time: 1min 53s


In [11]:
df.head(1)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq,comment_like_total,morphs
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201.0,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138267,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859,138599,햇살 우릴 위해 내리다 바람 서 롤 감싸다 하다 우리 웃음 속 계절 오다 또 가다 ...


In [40]:
df['total'] = df.morphs + (' ' + df.title) + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 2

In [41]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)
indices = pd.Series(df.index, index=df.songId)

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [50]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(30).index
    return df.songId.iloc[song_indices]

In [65]:
find_songid = '4083218'
a = get_recommendation(find_songid, cosine_sim)

In [66]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
sim_scores = pd.Series(cosine_sim_total[indices[find_songid]])
sim_scores.sort_values(ascending=False).head(11).tail(10)

1954    0.172005
1489    0.166212
1719    0.148663
1732    0.148641
1843    0.144897
1338    0.139385
1372    0.135915
1593    0.134953
1855    0.129901
1337    0.127008
dtype: float64

In [67]:
sorted_numbers = np.sort(df['comment_like_total'])
sorted_numbers

array([     1,      2,      2, ..., 457930, 471132, 556783], dtype=int64)

In [68]:
q1 = np.percentile(sorted_numbers, 25) # 25번째 백분위수를 계산하여 반환
q1

1410.5

In [69]:
q2 = np.percentile(sorted_numbers, 50)
q2

11876.0

In [70]:
# 25번째 백분위보다 크고 50번째 백분위보다 작은 songlist
filtered_data = df[(df['comment_like_total'] >= q1) & (df['comment_like_total'] < q2)]
print(filtered_data.shape)
filtered_data.head(1)

(1561, 16)


,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq,comment_like_total,morphs,total
9,35837631,편지 (영화 '동감' X 윤하 (YOUNHA)),윤하 (YOUNHA),발라드 국내영화,편지 (영화 '동감' X 윤하 (YOUNHA)),허승경,김광진,20221116.0,https://cdnimg.melon.co.kr/cm2/album/images/11...,67,10275,여기까지가 끝인가 보오\n이제 나는 돌아서겠소\n억지 노력으로 인연을 거슬러\n괴롭...,496963017 456209348,10342,여기 까지가 끝 인가 보오 이제 나 돌아서다 억지 노력 으로 인연 거슬러 괴롭히다 ...,여기 까지가 끝 인가 보오 이제 나 돌아서다 억지 노력 으로 인연 거슬러 괴롭히다 ...


In [71]:
get_recommendation(find_songid, cosine_sim)

1330     4083218
1954       81122
1489     4024086
1719     2669367
1732     1125952
1843     1125932
1338     1003566
1372     1749951
1593     1925230
1855     3621929
1337     3622464
1518     1124649
1411      833074
632      4132144
5468    35029957
1429      833077
4967    30384697
2232     3047051
1698     2467820
2865     1563237
1865     1003056
5892    34052428
1396     1124998
1486     1573393
1960       60162
5337    34374826
1802     1054752
81       3515044
889      4365747
204      2719448
Name: songId, dtype: object

In [17]:
[i for i in a.values if i in filtered_data.songId.values]

['4185941',
 '58506',
 '1053842',
 '4260411',
 '33461694',
 '33507532',
 '33435717',
 '3592201',
 '1099824']

In [78]:
a = get_recommendation(find_songid, cosine_sim)
b = [i for i in a.values if i in filtered_data.songId.values]
filtered = df[df['songId'].isin(b[:5])].to_dict('records')

In [79]:
filtered

[{'songId': '3515044',
  'title': 'Heaven',
  'artist': '다비치',
  'genre': '발라드 국내영화',
  'album': '기생령 OST',
  'lyricist': '준 (June)',
  'composer': '민명기',
  'date': 20110801.0,
  'img': 'https://cdnimg.melon.co.kr/cm/album/images/013/59/258/1359258_500.jpg/melon/resize/282/quality/80/optimize',
  'comment': 13,
  'like': 4481,
  'lyric': '아무 말도 말아요 그대 마음 알아요\n이제 나를 놓고 싶은거죠\n그댄 정말 바보죠 난 정말 괜찮아\n미안해요 보내주지 못해서\n바보같은 내 맘 다칠까\n헤어지잔 아픈 말 못한\n그대 더 아팠을테죠\n그래 헤어져요 그립고 그리워도\n익숙해질게요\n그대 없는 하루 또 하루\n하나만 기억해요\n나는 항상 이 자리에\n기다릴게요 언제라도 돌아와\n눈물 마를 날 없이 온종일 울어도\n괜찮아요 그대 행복하다면\n바보같은 내 맘 다칠까\n헤어지잔 아픈 말 못한\n그대 더 아팠을테죠\n그래 헤어져요 그립고 그리워도\n익숙해질게요\n그대 없는 하루 또 하루\n하나만 기억해요\n나는 항상 이 자리에\n기다릴게요 언제라도 돌아와\n우리 지난 시간을 돌려\n우리 사랑했던\n그때로 돌아가고 싶은데\n지난 꿈일테죠 괜한 욕심일테죠\n바보라고 해도 내가 아니라 해도\n기다릴거예요 그대 다시 오는날까지\n하나만 기억해요\n항상 여기 이 자리에\n내가 있어요 영원히 난 너뿐야',
  'plylstSeq': '402724097 434586240',
  'comment_like_total': 4494,
  'morphs': '아무 말 말다 그대 마음 알다 이제 나르다 놓다 싶다 대다 정말 바보 죠 난 정말 괜찮다 미안하다 보내다 못 하다 바보 같다 내 맘 다치다 헤어지다 아프다 말 못 한 그대 

In [ ]:
b = []
for i in a.values:
    if i in filtered_data.songId.values:
        b.append(i)
        

In [74]:
# 코사인 유사도
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)

In [75]:
cosine_sim_total[indices[find_songid]]

array([0.01609379, 0.04257023, 0.09403357, ..., 0.01969947, 0.03113656,
       0.01127102])

In [76]:
sim_scores = pd.Series(cosine_sim_total[indices[find_songid]])
sim_scores

0       0.016094
1       0.042570
2       0.094034
3       0.062950
4       0.016053
          ...   
6242    0.089257
6243    0.016817
6244    0.019699
6245    0.031137
6246    0.011271
Length: 6247, dtype: float64

In [77]:
# 나를 제외한 Top 10
sim_scores.sort_values(ascending=False).head(11).tail(10)

1954    0.172005
1489    0.166212
1719    0.148663
1732    0.148641
1843    0.144897
1338    0.139385
1372    0.135915
1593    0.134953
1855    0.129901
1337    0.127008
dtype: float64